In [4]:
import sklearn
import pickle
import numpy as np 
import torch 
from ClassificationModels.CNN_T import ResNetBaseline, get_all_preds, fit, UCRDataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
from tslearn.datasets import UCR_UEA_datasets
import os 

In [5]:
dataset='NATOPS'

In [6]:
X_train,y_train, X_test, y_test=UCR_UEA_datasets().load_dataset(dataset)
train_x=X_train.reshape(-1,X_train.shape[-1],X_train.shape[-2])
test_x=X_test.reshape(-1,X_train.shape[-1],X_train.shape[-2])
train_y = y_train
test_y=y_test



In [7]:
print(train_x.shape)
print(X_train.shape)

(180, 24, 51)
(180, 51, 24)


In [8]:
enc1=sklearn.preprocessing.OneHotEncoder(sparse=False).fit(train_y.reshape(-1,1))
pickle.dump(enc1,open(f'../ClassificationModels/models/{dataset}/OneHotEncoder.pkl','wb'))
train_y=enc1.transform(train_y.reshape(-1,1))
test_y=enc1.transform(test_y.reshape(-1,1))



In [9]:
n_pred_classes =train_y.shape[1]
print('n pred classes',n_pred_classes) 
train_dataset = UCRDataset(train_x.astype(np.float64),train_y.astype(np.int64))
test_dataset = UCRDataset(test_x.astype(np.float64),test_y.astype(np.int64))
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)


n pred classes 6


In [10]:
model = ResNetBaseline(in_channels= X_train.shape[-1], num_pred_classes=n_pred_classes)
fit(model,train_loader,test_loader)
if dataset in os.listdir('../ClassificationModels/models/'):
    print('Folder exists')
else: 
    os.mkdir(f'../ClassificationModels/models/{dataset}')
torch.save(model.state_dict(), f'../ClassificationModels/models/{dataset}/ResNet')

test_preds, ground_truth = get_all_preds(model, test_loader)
ground_truth=np.argmax(ground_truth,axis=1)
#test_preds=np.argmax(test_preds,axis=1)
sns.set(rc={'figure.figsize':(5,4)})
heatmap=confusion_matrix(ground_truth, test_preds)
sns.heatmap(heatmap, annot=True)
plt.savefig(f'../ClassificationModels/models/{dataset}/ResNet_confusion_matrix.png')
plt.close()
acc= accuracy_score(ground_truth, test_preds)
a = classification_report(ground_truth, test_preds, output_dict=True)
dataframe = pd.DataFrame.from_dict(a)
dataframe.to_csv(f'../ClassificationModels/models/{dataset}/classification_report.csv', index = False)



Epoch: 1, Train loss: 1.186, Val loss: 1.367
Epoch: 2, Train loss: 0.944, Val loss: 0.959
Epoch: 3, Train loss: 0.601, Val loss: 0.59
Epoch: 4, Train loss: 0.557, Val loss: 0.474
Epoch: 5, Train loss: 0.372, Val loss: 0.415
Epoch: 6, Train loss: 0.388, Val loss: 0.659
Epoch: 7, Train loss: 0.377, Val loss: 0.596
Epoch: 8, Train loss: 0.35, Val loss: 0.504
Epoch: 9, Train loss: 0.337, Val loss: 0.424
Epoch: 10, Train loss: 0.337, Val loss: 0.553
Epoch: 11, Train loss: 0.314, Val loss: 0.409
Epoch: 12, Train loss: 0.292, Val loss: 0.424
Epoch: 13, Train loss: 0.308, Val loss: 0.397
Epoch: 14, Train loss: 0.326, Val loss: 0.356
Epoch: 15, Train loss: 0.252, Val loss: 0.427
Epoch: 16, Train loss: 0.276, Val loss: 0.361
Epoch: 17, Train loss: 0.309, Val loss: 0.331
Epoch: 18, Train loss: 0.282, Val loss: 0.352
Epoch: 19, Train loss: 0.261, Val loss: 0.34
Epoch: 20, Train loss: 0.237, Val loss: 0.339
Epoch: 21, Train loss: 0.252, Val loss: 0.491
Epoch: 22, Train loss: 0.277, Val loss: 0.345


In [11]:
print(train_x.shape)
print(test_x.shape)
print(X_train.shape[-1])

(180, 24, 51)
(180, 24, 51)
24


# Explanation Algo

In [12]:
item=test_x[0].reshape(1,test_x.shape[1],-1)
shape=item.shape
_item=  torch.from_numpy(item).float()
model.eval()
#y_target= model(_item).detach().numpy()
print(model(_item))
y_target = torch.nn.functional.softmax(model(_item)).detach().numpy()

tensor([[ -2.8414, -44.8772, -88.3838,  92.3523, -28.3477,  -0.6938]],
       grad_fn=<AddmmBackward0>)


/home/jacqueline/.local/share/virtualenvs/TSInterpret-NXJYnQDU/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


In [13]:
print(y_target)

[[4.5486e-42 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 3.8957e-41]]


In [14]:
from TSInterpret.InterpretabilityModels.counterfactual.Ates import AtesCF
exp_model= AtesCF(model,(train_x,train_y),backend='PYT',mode='feat')


2022-06-27 15:53:04.039055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-27 15:53:04.039093: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
#TODO Target Calculation  --> Why is opt not working
exp = exp_model.explain(item,3, method= 'opt')


Run Brute Force as Backup.


In [16]:

array, label=exp


In [17]:
array

array([[[-0.597549, -1.897467, -0.689874, ..., -0.592066, -1.912729,
         -0.702529],
        [ 0.599973, -1.917021, -0.838037, ...,  0.597675, -1.918653,
         -0.842211],
        [-0.594716, -0.835395, -0.121241, ..., -0.599888, -0.82464 ,
         -0.119868],
        ...,
        [ 0.627078, -1.620468, -0.702724, ...,  0.76356 , -1.463721,
         -0.630478],
        [-0.767244, -1.756   , -0.682146, ..., -0.778381, -1.726662,
         -0.68518 ],
        [ 0.791354, -1.66717 , -0.84923 , ...,  0.670227, -1.707901,
         -0.787852]]])

# Plot Item 

In [18]:
i=0
org_label=0
cf_label=label[0]
exp=array

exp_model.plot(item,org_label,exp,cf_label,figsize=(15,15))

Items are identical
